<a href="https://colab.research.google.com/github/ricardoalmeidagrover/AI_Agent_Describe_Product/blob/main/AI_Agent_To_Describe_Product.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q google-genai google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [ ]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from google import genai

client = genai.Client()

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types

In [ ]:
session_service = InMemorySessionService()

In [ ]:
async def call_agent(agent: Agent, message_text: str) -> str:
    session = await session_service.create_session(app_name=agent.name, user_id="user1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    async for event in runner.run_async(user_id="user1", session_id=session.id, new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
for model in client.models.list():
    print(model.name)

models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-pro-preview-05-06
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview

In [ ]:
FAST_MODEL = "gemini-2.5-flash-preview-05-20"
PRO_MODEL = "gemini-2.5-flash-preview-04-17-thinking"

In [ ]:
##########################################
# --- Agent 1: Product finder --- #
##########################################

async def search_agent(product_name):

    agent = Agent(
        name="search_agent",
        model=FAST_MODEL,
        instruction="""
        You are an search assistent. Your task is to use google search tool (google_search)
        to find products in categories like 'phones and tables', 'computers', 'cameras', 'TV and projectors', etc.
        Based on the product name, list all product's variants, including EAN (European Article Number) if exists
        """,
        description="Search agent",
        tools=[google_search]
    )

    search_agent_input = f"Product: {product_name}"

    variants = await call_agent(agent, search_agent_input)
    return variants

In [ ]:
################################################
# --- Agent 2: Product Assets --- #
################################################

async def assets_agent(product_name, variants):
    agent = Agent(
        name="assets_agent",
        model=FAST_MODEL,
        instruction="""
        You are a Catalog Manager focused on finding images and videos for a
        specific products.
        You receive a list of products and uses Google Search (google_search) to
        find more images and videos that I can use in a Marketplace landing page.
        """,
        description="Assets agent",
        tools=[google_search]
    )

    assets = await call_agent(agent, variants)
    return assets

In [ ]:
################################################
# --- Agent 3: Product specifications --- #
################################################

async def specifications_agent(product_name, variants):
    agent = Agent(
        name="specifications_agent",
        model=FAST_MODEL,
        instruction="""
        You are a Catalog Manager focused on finding specifications and features
        for specific products.
        You receive a list of products and uses Google Search (google_search) to
        find more specifications and features of each product.
        """,
        description="Specifications agent",
        tools=[google_search]
    )

    specifications = await call_agent(agent, variants)
    return specifications

In [ ]:
################################################
# --- Agent 4: Product Content Editor --- #
################################################

async def content_editor_agent(product_name, variants, assets, specifications):
    agent = Agent(
        name="content_editor_agent",
        model=PRO_MODEL,
        instruction="""
        You are a Content Manager Editor focused on describing the product on
        a Marketplace landing page.
        You receive a list of product's variants, images and videos,
        Create an atracting description, highlight
        the main features in bullet points and describe the Technical data.
        """,
        description="Product Content Editor agent"
    )

    content_input = f"Product: {product_name}\nVariants: {variants}\nImages and videos: {assets}\nSpecifications: {specifications}"
    product_content = await call_agent(agent, variants)
    return product_content

In [ ]:
import textwrap
from IPython.display import display, Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
print("🚀 Starting the system to search product details 🚀\n")

product_name = input("❓ Please, type the PRODUCT NAME that you want to find details: ")

if not product_name:
    print("\nPlease type the product name!")
else:
    print(f"\Cool! Let's detail the product {product_name}")

    variants = await search_agent(product_name)
    print("\n--- 📝 Agent 1 Result (Search) ---\n")
    display(to_markdown(variants))
    print("--------------------------------------------------------------")

    assets = await assets_agent(product_name, variants)
    print("\n--- 📝 Agent 2 Result: (Assets) ---\n")
    display(to_markdown(assets))
    print("--------------------------------------------------------------")

    specifications = await specifications_agent(product_name, variants)
    print("\n--- 📝 Agent 3 Result (Specification) ---\n")
    display(to_markdown(specifications))
    print("--------------------------------------------------------------")

    product_content = await content_editor_agent(product_name, variants, assets, specifications)
    print("\n--- 📝 Agent 4 Result (Content Manager) ---\n")
    display(to_markdown(product_content))
    print("--------------------------------------------------------------")

🚀 Starting the system to search product details 🚀

❓ Please, type the PRODUCT NAME that you want to find details: Microsoft Surface Pro, Copilot+ PC, 13” Touchscreen, Snapdragon® X Plus
\Cool! Let's detail the product Microsoft Surface Pro, Copilot+ PC, 13” Touchscreen, Snapdragon® X Plus

--- 📝 Agent 1 Result (Search) ---



> The Microsoft Surface Pro, Copilot+ PC, 13” Touchscreen, featuring the Snapdragon® X Plus processor, is available in several variants, primarily differing in storage capacity and color.
> 
> Here are some of the product's known variants and their associated EANs (where available):
> 
> *   **Microsoft Surface Pro, Copilot+ PC, 13”, Snapdragon® X Plus, 16GB RAM, 256GB SSD, Platinum.**
>     *   EAN: 196388288946
> *   **Microsoft Surface Pro, Copilot+ PC, 13”, Snapdragon® X Plus, 16GB RAM, 512GB SSD, Black.**
>     *   EAN: 196388288922
> *   **Microsoft Surface Pro, Copilot+ PC, 13”, Snapdragon® X Plus, 16GB RAM, 256GB SSD, Device Only (11th Edition), Platinum.**
>     *   Model Number: ZHX-00001
> *   **Microsoft Surface Pro, Copilot+ PC, 13”, Snapdragon® X Plus, 16GB RAM, 512GB SSD, Device Only (11th Edition), Black.**
>     *   Model Number: ZHX00001
> 
> Other identified specifications across these variants include a 13-inch PixelSense Flow touchscreen with a 2880 x 1920 resolution and up to a 120Hz refresh rate. The Snapdragon® X Plus processor typically features 10 cores (Snapdragon X Plus X1P-64-100). Some configurations may also include an OLED display option. Available colors may include Platinum, Black, Dune, and Sapphire.


--------------------------------------------------------------

--- 📝 Agent 2 Result: (Assets) ---



> The Microsoft Surface Pro, Copilot+ PC, 13” Touchscreen, featuring the Snapdragon® X Plus processor, is available in various configurations, and a wealth of images and videos are available to showcase its features and design.
> 
> **General Product Visuals:**
> 
> You can find numerous images and videos that provide an overall view of the Microsoft Surface Pro, Copilot+ PC, 13”. These often highlight its 2-in-1 design, allowing it to function as both a tablet and a laptop with the addition of a detachable keyboard. The sleek design, thin profile, and the built-in kickstand are commonly featured in promotional and review content. Many visuals showcase the device in various orientations to demonstrate its flexibility.
> 
> **Key Features and Visual Demonstrations:**
> 
> *   **Display:** Images and videos frequently emphasize the 13-inch PixelSense Flow touchscreen with its 2880 x 1920 resolution and up to 120Hz refresh rate. Visuals of the optional OLED display are also available, showcasing its vibrant colors and deep blacks.
> *   **Snapdragon X Plus Processor:** While the processor itself isn't directly visual, many videos discuss its performance and efficiency, often demonstrating the device's speed and responsiveness in various tasks like video editing and multitasking. Some content may show benchmarks or UI fluidity as a result of the powerful NPU with 45 trillion operations per second (TOPS).
> *   **AI Capabilities (Copilot+ PC):** Videos often highlight the AI-powered features, such as Cocreator for generating images from text prompts, Recall for searching PC history, and Windows Studio Effects for enhanced video calls. Demonstrations of the dedicated Copilot key on the keyboard are also common.
> *   **Cameras:** The 10.5-megapixel rear-facing camera with autofocus and 4K video support, and the HD front-facing camera with an ultrawide field of view and AI enhancements, are frequently shown in use for photos, video calls, and demonstrations.
> *   **Connectivity and Ports:** Visuals show the two USB-C®/USB4® ports, supporting charging, data transfer, and DisplayPort 1.4a, as well as the Surface Connect port. The presence of Wi-Fi 7 and optional 5G connectivity is also often mentioned or demonstrated.
> *   **Accessories:** Many images and videos feature the Surface Pro with its compatible accessories, such as the Surface Pro Flex Keyboard (which can be used attached or detached via Bluetooth), Surface Pro Keyboard, and the Surface Slim Pen (2nd edition).
> 
> **Color Variants:**
> 
> Images are available for the various color options:
> 
> *   **Platinum:** Widely featured in official and review content.
> *   **Black:** Frequently shown as a core color option.
> *   **Dune:** Images and product listings confirm its availability.
> *   **Sapphire:** Images and product listings confirm its availability.
> *   **Violet/Ocean:** While not explicitly listed in the provided product variants, these colors are mentioned as new options for the Surface Laptop and Surface Pro in some official communications.
> 
> **Videos:**
> 
> Numerous video types are available:
> 
> *   **Official Product Videos:** Microsoft has released official videos showcasing the Surface Pro and its Copilot+ PC features.
> *   **Unboxing Videos:** Several creators and official channels have produced unboxing videos, providing a first look at the device and its packaging.
> *   **Reviews and Hands-on:** Many tech reviewers have published in-depth video reviews, testing the device's performance, battery life, AI capabilities, and overall user experience.
> *   **Feature Demonstrations:** Videos often specifically demonstrate new AI features like Recall, Cocreator, and Live Captions.
> 
> When searching for specific EANs or model numbers, you can find corresponding product images and specifications on retailer and manufacturer websites.


--------------------------------------------------------------

--- 📝 Agent 3 Result (Specification) ---



> The Microsoft Surface Pro, Copilot+ PC, 13” Touchscreen, featuring the Snapdragon® X Plus processor (10-core, model X1P-64-100), is a 2-in-1 device designed for portability and AI-accelerated experiences. It is also referred to as the Surface Pro 11th Edition.
> 
> Here's a detailed breakdown of its specifications and features:
> 
> **Display:**
> *   **Screen Size:** 13-inch PixelSense™ Flow touchscreen.
> *   **Resolution:** 2880 x 1920 pixels (267 PPI).
> *   **Aspect Ratio:** 3:2.
> *   **Refresh Rate:** Dynamic refresh rate up to 120Hz.
> *   **Contrast Ratio:** 1200:1 for LCD, and 1,000,000:1 for OLED.
> *   **Color Profile:** sRGB and Vivid.
> *   **Brightness:** SDR: 600 nits maximum (typical) for both LCD and OLED; HDR: 600 nits peak luminance for LCD and 900 nits peak luminance for OLED.
> *   **Technology:** Features adaptive color, adaptive contrast, auto color management, 10-point multi-touch, Dolby Vision IQ™ support, and Corning® Gorilla® Glass 5.
> *   **Display Options:** Available with either an LCD or an optional OLED display.
> 
> **Processor and AI Capabilities:**
> *   **CPU:** Qualcomm® Snapdragon® X Plus (10-core, model X1P-64-100). Some configurations may also offer the Snapdragon® X Elite (12-core) processor.
> *   **GPU:** Integrated Qualcomm® Adreno™ GPU.
> *   **NPU:** Qualcomm® Hexagon™ Neural Processing Unit (NPU) with 45 trillion operations per second (TOPS), enabling AI-powered Copilot+ PC experiences.
> 
> **Memory and Storage:**
> *   **RAM:** 16GB LPDDR5x RAM, with options for 32GB RAM also available.
> *   **Storage:** Removable Gen 4 Solid State Drive (SSD) with capacities including 256GB, 512GB, and 1TB.
> 
> **Cameras and Audio:**
> *   **Front Camera:** Quad HD front-facing Surface Studio Camera (1440p) with an ultrawide field of view. It includes Windows Studio Effects like automatic framing, Portrait Blur, Creative Filters, Eye Contact: Teleprompter, and Portrait Light.
> *   **Rear Camera:** 10 MP Ultra HD rear-facing camera.
> *   **Audio:** 2W stereo speakers with Dolby® Atmos® and Dual Studio Mics with voice focus.
> 
> **Ports and Connectivity:**
> *   **USB Ports:** Two USB-C® / USB4® ports with support for charging, data transfer, DisplayPort 1.4a, and Surface Thunderbolt™ 4 Dock and other accessories.
> *   **Charging:** Supports fast charging with a minimum 65W power supply via Surface Connect or USB-C.
> *   **Other Ports:** Surface Pro Keyboard port and Surface Connect port.
> *   **Wireless:** Wi-Fi 7 (802.11be) and Bluetooth 5.4. Optional 5G connectivity is available, particularly for business models.
> 
> **Battery Life:**
> *   Up to 14 hours of local video playback. Some sources indicate up to 20 hours for video playback.
> 
> **Design and Dimensions:**
> *   **Dimensions:** 11.3 inches (287 mm) L x 8.2 inches (209 mm) W x 0.37 inches (9.3 mm) H.
> *   **Weight:** 1.97 lbs (895 g).
> *   **Material:** Anodized aluminum casing.
> *   **Kickstand:** Iconic 165-degree fluid kickstand for flexible positioning.
> 
> **Operating System and Features:**
> *   **Operating System:** Windows 11 Home.
> *   **Security:** Enhanced security with Microsoft Pluton TPM 2.0 and Windows Hello face authentication with Enhanced Sign-in security.
> *   **Copilot+ PC Features:** Includes AI-powered experiences like Recall (search across time for content), Live Captions (understand new languages instantly), and Cocreator (generate artwork from text prompts).
> *   **Accessories Compatibility:** Compatible with optional Surface keyboards (including the Surface Pro Flex Keyboard) and Surface Slim Pen (sold separately).
> 
> **Available Colors:**
> *   Platinum, Black, Dune, and Sapphire.


--------------------------------------------------------------

--- 📝 Agent 4 Result (Content Manager) ---



> Okay, here is a compelling marketplace product description for the Microsoft Surface Pro, Copilot+ PC with Snapdragon X Plus, based on the information provided.
> 
> ---
> 
> **Headline:** Meet the New Microsoft Surface Pro, Copilot+ PC | 13" Touchscreen | Snapdragon X Plus | Unleash AI-Enhanced Performance & 2-in-1 Versatility
> 
> **Product Description:**
> 
> Experience the future of personal computing with the all-new Microsoft Surface Pro, re-imagined as a Copilot+ PC. This incredibly versatile 2-in-1 device is engineered for speed, efficiency, and next-generation AI experiences, powered by the cutting-edge Snapdragon® X Plus processor.
> 
> Seamlessly transition between a powerful laptop, a creative studio, and a portable tablet. The stunning 13-inch PixelSense Flow touchscreen brings your ideas to life with vibrant colors, sharp detail, and smooth motion, featuring high resolution and a dynamic refresh rate up to 120Hz. Tackle demanding tasks, enjoy immersive entertainment, and create with ease, all while benefiting from exceptional battery life designed to keep pace with your day.
> 
> With Copilot+ PC capabilities, integrated AI accelerates your workflow, sparks your creativity, and enhances your productivity like never before. From intelligent search and content creation assistance to powerful on-device AI features, the Surface Pro is built for the AI era.
> 
> Available in elegant finishes like Platinum and Black, with various storage options to suit your needs, the Surface Pro is your ultimate portable powerhouse for work, creation, and play.
> 
> **Key Features:**
> 
> *   **Copilot+ PC Intelligence:** Unlock next-level productivity and creativity with dedicated AI acceleration from the Neural Processing Unit (NPU), enabling powerful on-device AI experiences.
> *   **Powered by Snapdragon® X Plus:** Experience breakthrough performance and incredible efficiency with the advanced 10-core Snapdragon® X Plus processor, designed for demanding tasks and long battery life.
> *   **Stunning 13" PixelSense Flow Display:** Immerse yourself in a vibrant, high-resolution (2880 x 1920) touchscreen with a dynamic refresh rate up to 120Hz for silky-smooth visuals. Optional OLED display available for unparalleled contrast and color.
> *   **Ultimate Versatility:** Effortlessly switch between Laptop, Tablet, and Studio modes with the iconic built-in kickstand and optional keyboard.
> *   **Speed and Storage:** Multitask with ease thanks to 16GB of high-speed RAM. Store your files, apps, and projects on fast SSD storage (available in 256GB, 512GB, and other options).
> *   **Designed for Mobility:** A thin, light, and premium design paired with exceptional battery life makes it perfect for on-the-go productivity and entertainment.
> *   **Premium Finishes:** Choose from sophisticated colors including Platinum and Black (Dune and Sapphire potentially available on some configurations).
> 
> **Technical Data:**
> 
> *   **Display:** 13-inch PixelSense Flow Touchscreen
>     *   Resolution: 2880 x 1920
>     *   Refresh Rate: Up to 120Hz (Dynamic)
>     *   Panel Type: LCD / Optional OLED
> *   **Processor:** Qualcomm® Snapdragon® X Plus (X1P-64-100)
>     *   CPU Cores: 10
>     *   Integrated Graphics: Qualcomm® Adreno™ GPU
>     *   Neural Processing Unit (NPU): Yes, for Copilot+ AI acceleration
> *   **RAM:** 16GB LPDDR5X (standard on listed configurations)
> *   **Storage:** Solid State Drive (SSD)
>     *   Available Capacities: 256GB, 512GB (other options may exist)
> *   **Operating System:** Windows 11 Home (specific edition may vary by configuration)
> *   **Cameras:** Front and rear cameras (specific resolution may vary)
> *   **Audio:** Integrated speakers and microphones
> *   **Connectivity:** Wi-Fi 7, Bluetooth 5.4, 2x USB-C® ports (type may vary by configuration), Surface Connect port.
> *   **Color Options (based on variants):** Platinum, Black. (Dune, Sapphire potentially available on some configurations).
> *   **Dimensions & Weight:** (Specific dimensions and weight vary slightly by configuration - typically ultra-portable)
> *   **Battery Life:** Long-lasting battery (up to [Manufacturer Estimated Hours] based on typical usage - *Note: Specific hours often provided by manufacturer*)
> 
> ---
> 
> **Note:** Accessories such as the Surface Pro Keyboard and Surface Slim Pen are typically sold separately, especially with "Device Only" configurations. Available colors and storage options may vary by specific product variant. Please check the variant details (EAN/Model Number) for exact specifications.


--------------------------------------------------------------
